# Φ-OS: General Purpose Optical Compiler
### Proof of Utility: Compiling a Universal NAND Gate

This notebook demonstrates the advanced capabilities of the `VirtualHolographicPU` (VPU) by compiling a universal NAND gate. This moves beyond a simple proof-of-concept to a proof of utility, showcasing the system's ability to create fundamental building blocks of computation.

We will:
1. Define the VPU with a 2-input, 1-output port configuration.
2. Specify the logic for a NAND gate using a formal truth table.
3. Invoke the new gradient-based compiler to solve for the physical substrate (`Δn`).
4. Verify the compiled gate's digital accuracy.
5. Visualize the resulting physical structure and the energy flow for a specific case.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from src.phi_cloud_core.vpu import VirtualHolographicPU, PhiSubstrate

print("Libraries loaded successfully.")

## Step 1: Define VPU Architecture and Logic

We instantiate a `PhiSubstrate` which represents our physical medium. We then configure the `VirtualHolographicPU` with two input ports and one output port. The `truth_table` for a NAND gate is defined, where the output is `0` only when both inputs are `1`.

In [ ]:
# Define the physical substrate for our computation
substrate = PhiSubstrate(shape=(256, 1024), dx=0.2e-6)

# Configure a VPU with 2 inputs and 1 output
vpu = VirtualHolographicPU(
    substrate=substrate,
    input_ports=[64, 192],  # Two inputs, spaced apart
    output_ports=[128],       # One output in the center
    port_width=10
)

# Define the NAND gate logic
nand_truth_table = [
    ([0, 0], [1]),
    ([0, 1], [1]),
    ([1, 0], [1]),
    ([1, 1], [0])
]

print("VPU configured for NAND gate compilation.")

## Step 2: Compile and Verify

We invoke the `learn_function`, which uses gradient descent to find an optimal `Δn` that satisfies the NAND logic. After compilation, we call `verify` to confirm that the analog energy outputs correctly map to the digital truth table.

In [ ]:
delta_n_final, loss_history = vpu.learn_function(nand_truth_table)

is_verified = vpu.verify(nand_truth_table)

## Step 3: Visualize the Compiled Hardware and Logic Flow

Finally, we visualize the results. The first plot shows the loss function decreasing over epochs, indicating learning is occurring. The second plot shows the final, compiled refractive index structure (`Δn`). The third plot visualizes the wave intensity for the `(1, 1) -> 0` case, demonstrating how the compiled structure correctly routes the light to produce a low-energy (logic '0') output.

In [ ]:
# Execute the [1, 1] case to get the fields for visualization
vpu.execute([1, 1], store_fields=True)
intensity_map = np.abs(np.array(vpu.substrate.forward_fields))**2

# Create plots
fig, axes = plt.subplots(1, 3, figsize=(21, 6))
fig.patch.set_facecolor('#1a1a1a')

# 1. Loss History
axes[0].plot(loss_history, color='cyan')
axes[0].set_title("Loss History", color='white')
axes[0].set_xlabel("Epoch", color='white')
axes[0].set_ylabel("Mean Squared Error", color='white')
axes[0].set_yscale('log')
axes[0].tick_params(colors='white')

# 2. Compiled Substrate
im = axes[1].imshow(delta_n_final.T, aspect='auto', cmap='viridis', origin='lower')
axes[1].set_title(f"Compiled NAND Gate (Verified: {is_verified})", color='white')
axes[1].set_xlabel("x position", color='white')
axes[1].set_ylabel("z position (propagation)", color='white')
axes[1].tick_params(colors='white')
fig.colorbar(im, ax=axes[1], label='Δn')

# 3. Execution for Input [1, 1]
im_exec = axes[2].imshow(intensity_map.T, aspect='auto', cmap='inferno', origin='lower')
axes[2].set_title("Execution: Input [1, 1] -> Output [0]", color='white')
axes[2].set_xlabel("x position", color='white')
axes[2].set_ylabel("z position (propagation)", color='white')
axes[2].tick_params(colors='white')
fig.colorbar(im_exec, ax=axes[2], label='Intensity')

plt.tight_layout()
plt.show()